In [4]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers


In [5]:
# Set your data path
data_path = "D:/Curated X-Ray Dataset/"

# Function to load and preprocess images
def load_images(folder):
    images = []
    labels = []
    for label, class_name in enumerate(['Normal', 'Pneumonia-Bacterial', 'Pneumonia-Viral', 'COVID-19']):
        class_path = os.path.join(folder, class_name)
        for filename in os.listdir(class_path):
            img = cv2.imread(os.path.join(class_path, filename), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (224, 224))  # Resize images to match DenseNet input size
            img = img / 255.0  # Normalize pixel values to [0, 1]
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

# Load images and labels
images, labels = load_images(data_path)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

# Apply oversampling to handle class imbalance
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train.reshape(-1, 224 * 224), y_train)
X_train_resampled = X_train_resampled.reshape(-1, 224, 224, 1)

# Split the resampled data into training and validation sets
X_train_resampled, X_val_resampled, y_train_resampled, y_val_resampled = train_test_split(
    X_train_resampled, y_train_resampled, test_size=0.2, random_state=42, stratify=y_train_resampled
)

X_train_resampled = np.repeat(X_train_resampled, 3, axis=-1)
X_val_resampled = np.repeat(X_val_resampled, 3, axis=-1)
X_test = np.repeat(X_test, 3, axis=-1)

In [6]:

# Create a DenseNet model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')  # Adjust the number of output nodes according to your classes
])

# Compile the model
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
# # Data augmentation
# train_datagen = ImageDataGenerator(
#     rotation_range=15,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# train_generator = train_datagen.flow(X_train_resampled, y_train_resampled, batch_size=32)

# # Train the model without data augmentation
# history = model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32,
#                     validation_data=(X_val_resampled, y_val_resampled))

# # Evaluate the model on the test set
# test_loss, test_acc = model.evaluate(X_test, y_test)
# print(f'Test Accuracy: {test_acc * 100:.2f}%')


# Train the model
history = model.fit(X_train_resampled, y_train_resampled, epochs=10, validation_data=(X_val_resampled, y_val_resampled))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4376710560884668783
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 923205632
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4652280111060714548
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
